In [1]:

# Cell
#export
import os
import pandas as pd
import numpy as np
import plotly.express as px

# Cell
def load_bt(DATA_RAW, stock_id, train_or_test='train'):
    """Loads the book and trade data into a single dataframe."""
    book = pd.read_parquet(os.path.join(DATA_RAW, f'book_{train_or_test}.parquet/stock_id={stock_id}'))
    trade =  pd.read_parquet(os.path.join(DATA_RAW, f'trade_{train_or_test}.parquet/stock_id={stock_id}'))
    return book.merge(trade, on=['time_id', 'seconds_in_bucket'], how='outer')

# Cell
def add_wap(df):
    """Adds the weighted average price to a book df."""
    df['wap'] = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1']+ df['ask_size1'])
    return df

# Cell
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

# Cell
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

# Cell
def realized_volatility_per_time_id(file_path, prediction_column_name):
    df_book_data = pd.read_parquet(file_path)
    df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
                                      df_book_data['bid_size1']+ df_book_data['ask_size1'])
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    stock_id = file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    return df_realized_vol_per_stock[['row_id',prediction_column_name]]

# Cell
def past_realized_volatility_per_stock(list_file,prediction_column_name):
    df_past_realized = pd.DataFrame()
    for file in list_file:
        df_past_realized = pd.concat([df_past_realized,
                                     realized_volatility_per_time_id(file,prediction_column_name)])
    return df_past_realized

# Cell
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def make_folds(DATA_RAW, DATA_INTERIM=None):
    train = pd.read_csv(os.path.join(DATA_RAW, 'train.csv'))
    g = GroupKFold()
    for fold, (_, test_idx) in enumerate(g.split(train, groups=train['time_id'])):
        train.loc[test_idx, 'fold'] = fold
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    if not DATA_INTERIM: return train
    train.to_csv(os.path.join(DATA_INTERIM, 'folds.csv'), index=False)
    
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

def mean_decay(x, decay=.9, step=-1, axis=1): 
    """Returns sum with exponential decay, step = -1
    for the end of the array to matter the most."""
    
    weights = np.power(decay, np.arange(x.shape[axis])[::step]).astype(np.float32)
    return np.sum(weights * x, axis=axis)

In [2]:
import os
from itertools import product
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
from fastcore.script import call_parse
import glob
import plotly.express as px

In [3]:
DATA_RAW = '../input/optiver-realized-volatility-prediction'
DATA_INTERIM = ''

In [4]:
def make_fe1(DATA_RAW,train_or_test):
    
    if train_or_test == 'test': 
        df_fe = pd.read_csv(os.path.join(DATA_RAW, 'test.csv')).set_index('row_id')
    else: 
        df_fe = make_folds(DATA_RAW).set_index('row_id')
    df_all = pd.DataFrame()
    
    for stock_id in df_fe.stock_id.unique():
        df = load_bt(DATA_RAW, stock_id, train_or_test)
        df = add_wap(df)
        df['log_return'] = df.groupby(['time_id'])['wap'].apply(log_return)
        df['abs_log_return'] = df['log_return'].abs()
        df['is_pos_return'] = df['log_return'] > 0
        df['is_neg_return'] = df['log_return'] < 0
        df['minute'] = np.ceil((df.seconds_in_bucket + .1) / 120).astype(int)

        df_agg = pd.DataFrame()
        df_agg['real_vol'] = df.groupby('time_id')['log_return'].agg(realized_volatility)
        df_agg['is_pos_return_sum'] = df.groupby('time_id')['is_pos_return'].agg(sum).astype(np.float32)
        df_agg['is_neg_return_sum'] = df.groupby('time_id')['is_neg_return'].agg(sum).astype(np.float32)

        ############ Realized volume for each minute ############
        for minute in df.minute.unique():
            tmp = df[df['minute'] == minute]
            df_agg[f'real_vol_min_{minute}'] = tmp.groupby('time_id')['log_return'].agg(realized_volatility)

        ######### Decay sum of realized volume per minute ########
        cols = [f'real_vol_min_{minute}' for minute in df.minute.unique()]
        x = df_agg[cols].values
        for decay, step in product((.99, .95, .9, .85), (1, -1)): 
            df_agg[f'real_vol_mean_decay_{decay}_{step}'] =  mean_decay(x, decay, step)
        df_agg['end_beg_decay_ratio'] = df_agg['real_vol_mean_decay_0.85_-1'] / df_agg['real_vol_mean_decay_0.85_1']

        ################# Adding 'row_id' column ##################
        df_agg.reset_index(inplace=True)
        df_agg['time_id'] = df_agg.time_id.apply(lambda x: f"{stock_id}-{x}")
        df_agg.rename({'time_id': 'row_id'}, axis=1, inplace=True)

        ############### Add df_agg to all features #################
        df_all = pd.concat([df_all, df_agg])

    ################### Merge with train or test file ###############
    return df_fe.join(df_all.set_index('row_id')).reset_index()

In [5]:
fe1_train = make_fe1(DATA_RAW, 'train')
fe1_train.to_pickle('fe1_train.pkl')
fe1_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428932 entries, 0 to 428931
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   row_id                       428932 non-null  object 
 1   stock_id                     428932 non-null  int64  
 2   time_id                      428932 non-null  int64  
 3   target                       428932 non-null  float64
 4   fold                         428932 non-null  float64
 5   real_vol                     428932 non-null  float64
 6   is_pos_return_sum            428932 non-null  float32
 7   is_neg_return_sum            428932 non-null  float32
 8   real_vol_min_1               428932 non-null  float64
 9   real_vol_min_2               428932 non-null  float64
 10  real_vol_min_3               428931 non-null  float64
 11  real_vol_min_4               428931 non-null  float64
 12  real_vol_min_5               428931 non-null  float64
 13 

In [6]:
####################### Ideas not used yet ######################
# df['spread_pct'] = (df.ask_price1 - df.bid_price1) / df.wap
# df['spread_2_pct'] = (df.ask_price2 - df.bid_price2) / df.wap
# df['spread'] = (df.ask_price1 - df.bid_price1) 
# df['spread_2'] = (df.ask_price2 - df.bid_price2) 
# df['sum_bid'] = df[['bid_size1', 'bid_size2']].sum(axis=1)
# df['sum_ask'] = df[['ask_size1', 'ask_size2']].sum(axis=1)
# df['sum_bid_ask'] = df[['bid_size1', 'ask_size1', 'bid_size2', 'ask_size2']].sum(axis=1)
# df['bid_ask_ratio'] = df['sum_bid'] / df['sum_ask']